In [1]:
# Environment & Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports ready!")
print(f"Pandas: {pd.__version__}, NumPy: {np.__version__}")

✅ Imports ready!
Pandas: 2.1.4, NumPy: 1.24.3


In [7]:
# Install & Load from HF Dataset
!pip install datasets -q
from datasets import load_dataset


You should consider upgrading via the '/Users/fl_lpt-759/AJ-codes/AJ-DBA/visit-with-us-mlops/venv/bin/python3 -m pip install --upgrade pip' command.


In [8]:
# Load directly from YOUR HF Dataset
dataset = load_dataset("AnjaneyaluAJ/wellness-tourism-data")
df = dataset['train'].to_pandas()  # Convert to pandas

print("✅ Loaded from HF Dataset!")
print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
df.head(2)

raw_wellness_package.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4128 [00:00<?, ? examples/s]

✅ Loaded from HF Dataset!
Shape: (4128, 21)

Columns: ['Unnamed: 0', 'CustomerID', 'ProdTaken', 'Age', 'TypeofContact', 'CityTier', 'DurationOfPitch', 'Occupation', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'ProductPitched', 'PreferredPropertyStar', 'MaritalStatus', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore', 'OwnCar', 'NumberOfChildrenVisiting', 'Designation', 'MonthlyIncome']


,Unnamed: 0,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,...,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,...,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,...,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0


In [9]:
# Remove unnecessary columns
cols_to_drop = ['Unnamed: 0', 'CustomerID']
df_clean = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

# Fix data types
numeric_cols = ['Age', 'DurationOfPitch', 'PreferredPropertyStar', 
                'NumberOfTrips', 'NumberOfChildrenVisiting']
for col in numeric_cols:
    if col in df_clean.columns:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce').fillna(0).astype(int)

# Handle missing values
df_clean['MonthlyIncome'] = df_clean['MonthlyIncome'].fillna(df_clean['MonthlyIncome'].median())
df_clean = df_clean.fillna('Unknown')

print("✅ Cleaned dataset:")
print("Shape:", df_clean.shape)
print("Missing values:", df_clean.isnull().sum().sum())
print("\nData types:")
df_clean.dtypes.value_counts()


✅ Cleaned dataset:
Shape: (4128, 19)
Missing values: 0

Data types:


int64      11
object      6
float64     2
Name: count, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

# Features & Target
X = df_clean.drop('ProdTaken', axis=1)
y = df_clean['ProdTaken']

# 80/20 stratified split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, stratify=y)

# Combine back
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Save locally
train_df.to_csv('../data/train_wellness.csv', index=False)
test_df.to_csv('../data/test_wellness.csv', index=False)

print("✅ Train/Test Split:")
print(f"Train: {train_df.shape} | Test: {test_df.shape}")
print(f"Train target balance: {train_df['ProdTaken'].value_counts(normalize=True).round(3)}")


✅ Train/Test Split:
Train: (3302, 19) | Test: (826, 19)
Train target balance: ProdTaken
0    0.807
1    0.193
Name: proportion, dtype: float64


In [15]:
# Push processed datasets to HF
from huggingface_hub import HfApi
import os

# Login (replace with your HF token from https://huggingface.co/settings/tokens)
from huggingface_hub import login
login("")  # Get from HF settings → Access Tokens

# Upload train/test
api = HfApi()
api.upload_file(
    path_or_fileobj="../data/train_wellness.csv",
    path_in_repo="train_wellness.csv",
    repo_id="AnjaneyaluAJ/wellness-tourism-data",
    repo_type="dataset"
)

api.upload_file(
    path_or_fileobj="../data/test_wellness.csv", 
    path_in_repo="test_wellness.csv",
    repo_id="AnjaneyaluAJ/wellness-tourism-data",
    repo_type="dataset"
)

print("✅ Train/Test uploaded to HF Dataset!")


✅ Train/Test uploaded to HF Dataset!


In [16]:
!pip install xgboost mlflow datasets scikit-learn -q
import xgboost as xgb
import mlflow
import mlflow.xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, roc_auc_score
from datasets import load_dataset

You should consider upgrading via the '/Users/fl_lpt-759/AJ-codes/AJ-DBA/visit-with-us-mlops/venv/bin/python3 -m pip install --upgrade pip' command.


In [17]:
# Load train/test from YOUR HF Dataset
dataset = load_dataset("AnjaneyaluAJ/wellness-tourism-data")
train_df = dataset['train'].to_pandas() if 'train' in dataset else pd.read_csv('data/train_wellness.csv')
test_df = dataset['test'].to_pandas() if 'test' in dataset else pd.read_csv('data/test_wellness.csv')

print("✅ Train/Test loaded!")
print(f"Train: {train_df.shape}, Test: {test_df.shape}")

train_wellness.csv: 0.00B [00:00, ?B/s]

test_wellness.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/3302 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/826 [00:00<?, ? examples/s]

✅ Train/Test loaded!
Train: (3302, 19), Test: (826, 19)


In [18]:
# Separate features/target
X_train = train_df.drop('ProdTaken', axis=1)
y_train = train_df['ProdTaken']
X_test = test_df.drop('ProdTaken', axis=1) 
y_test = test_df['ProdTaken']

# Encode categoricals
le_dict = {}
for col in X_train.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col].astype(str))
    X_test[col] = le.transform(X_test[col].astype(str))
    le_dict[col] = le

print("✅ Features encoded!")
print("X_train shape:", X_train.shape)


✅ Features encoded!
X_train shape: (3302, 18)


In [21]:
import joblib
import json
import os
os.makedirs('models', exist_ok=True)
print("models/ folder created")

# Model parameters grid (simple hyperparam tuning)
param_grid = [
    {'max_depth': 6, 'learning_rate': 0.1, 'n_estimators': 200, 'subsample': 0.8},
    {'max_depth': 4, 'learning_rate': 0.2, 'n_estimators': 100, 'subsample': 1.0},
    {'max_depth': 8, 'learning_rate': 0.05, 'n_estimators': 300, 'subsample': 0.8}
]

best_auc = 0
best_model = None
best_params = {}

print("🔍 Hyperparameter tuning...")
for i, params in enumerate(param_grid):
    print(f"Trial {i+1}: {params}")
    
    # Train model
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',
        random_state=42,
        **params
    )
    model.fit(X_train, y_train)
    
    # Evaluate
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    
    print(f"  AUC: {auc:.4f}")
    
    # Track best
    if auc > best_auc:
        best_auc = auc
        best_model = model
        best_params = params

# Save experiment log
experiment_log = {
    'best_auc': float(best_auc),
    'best_params': best_params,
    'trials': [{'params': p, 'auc': a} for p, a in zip(param_grid, [roc_auc_score(y_test, xgb.XGBClassifier(**p, random_state=42).fit(X_train, y_train).predict_proba(X_test)[:,1]) for p in param_grid])]
}

with open('models/experiment_log.json', 'w') as f:
    json.dump(experiment_log, f)

print(f"\n✅ Best Model - AUC: {best_auc:.3f}")
print(f"Params: {best_params}")

# Final evaluation
y_pred = best_model.predict(X_test)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


models/ folder created
🔍 Hyperparameter tuning...
Trial 1: {'max_depth': 6, 'learning_rate': 0.1, 'n_estimators': 200, 'subsample': 0.8}
  AUC: 0.9601
Trial 2: {'max_depth': 4, 'learning_rate': 0.2, 'n_estimators': 100, 'subsample': 1.0}
  AUC: 0.9186
Trial 3: {'max_depth': 8, 'learning_rate': 0.05, 'n_estimators': 300, 'subsample': 0.8}
  AUC: 0.9712

✅ Best Model - AUC: 0.971
Params: {'max_depth': 8, 'learning_rate': 0.05, 'n_estimators': 300, 'subsample': 0.8}

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.99      0.96       667
           1       0.92      0.76      0.83       159

    accuracy                           0.94       826
   macro avg       0.93      0.87      0.90       826
weighted avg       0.94      0.94      0.94       826



In [23]:
joblib.dump(best_model, '../models/best_wellness_xgb.pkl')
joblib.dump(le_dict, '../models/label_encoders.pkl')
print("Models saved successfully")
print(f"Best AUC logged: {best_auc:.4f}")

Models saved successfully
Best AUC logged: 0.9712
